In [1]:
import pandas as pd

df=pd.read_csv("./Data/Low2.csv")
df
# 열 Q, K1, K2, K3, K4의 내용을 합쳐서 하나의 열로 합치기
df['QK'] = df['Q'] +","+ df['K1'] +","+  df['K2'] + ","+ df['K3'] +","+  df['K4']
df=df[['QK','A']]
#열 이름을 text, prediction으로 고치기
df.columns=['text','prediction']
df


,text,prediction
0,"写真帳」を英語でいうと何 ?,loose leaf,account book,nOte,album",album
1,"鮭の内臓をとって、塩漬けにしたものを何という ?,さらまき鮭,はらまき鮭,からまき鮮,あらまき鮭",あらまき鮭
2,"次のうち、約21kmの距離で行われるマラソン競技はどれ ? _,フルマフソン,ハーフマラフ...",ハーフマラフソン
3,"秘の暑い時に、風を起こすために用いるものはどれ ?2,うちきさ,うちわ,うちこ,うちみ",うちわ
4,"関門海峡を挟んで九州と面している山口県の都市はどこ ?,上関市,下関市,右関市""6,左関市",下関市
...,...,...
22555,"石田スイの漫画『東京喰種トーキョーグール』の舞台となっている国はとどこ?,オーストラリア,日...",日本
22556,"フジテレビのクイズ番組『ネプリーグ』」のボーナスステージといえば「何アドベンチャー」 2,ロ...",トロッコ
22557,"エアラインといえば、何の会証 ?,鉄道会計,タクシー会視,航空会計,バス会i\r\nさス会証",航空会計
22558,"長崎に鳴滝塾を開いたシーボルトは円の ーー,中国人,ドイツ人,アフリカ人,サモア人",ドイツ人


In [2]:
import numpy as np
from datasets import load_dataset


train, valid, test = np.split(
    df.sample(frac=1, random_state=42), [int(0.99 * len(df)), int(0.999 * len(df))]
)

print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

Source News : 次のうち、空気中で最も速く移動すとで/ぁ】ち円肝/,イチローのバックホーム,リニアモーターカー,光,と=コ
四
Summarization : 光
Training Data Size : 22334
Validation Data Size : 203
Testing Data Size : 23


In [3]:
import torch
from transformers import T5Tokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence


def make_dataset(data, tokenizer, device):
    source = tokenizer(
        text=data.text.tolist(),
        padding="max_length",
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt"
    )

    target = tokenizer(
        text=data.prediction.tolist(),
        padding="max_length", 
        max_length=128,
        pad_to_max_length=True,
        truncation=True,
        return_tensors="pt"
    )
    
    source_ids = source["input_ids"].squeeze().to(device)
    source_mask = source["attention_mask"].squeeze().to(device)
    target_ids = target["input_ids"].squeeze().to(device)
    target_mask = target["attention_mask"].squeeze().to(device)
    return TensorDataset(source_ids, source_mask, target_ids, target_mask)

def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 1
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = T5Tokenizer.from_pretrained(
    pretrained_model_name_or_path="t5-small"
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(next(iter(train_dataloader)))
print(tokenizer.convert_ids_to_tokens(21603))
print(tokenizer.convert_ids_to_tokens(10))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[tensor([[   3,    2,    3,  ...,    0,    0,    0],
        [   3,    2, 5911,  ...,    0,    0,    0],
        [   3,    2,    3,  ...,    0,    0,    0],
        ...,
        [   3,    2,    3,  ...,    0,    0,    0],
        [   3,    2,  204,  ...,    0,    0,    0],
        [   3,    2,    3,  ...,    0,    0,    0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), tensor([[3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0],
        ...,
        [3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0],
        [3, 2, 1,  ..., 0, 0, 0]]), tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])]
▁summarize
:


In [4]:
from torch import optim
from transformers import T5ForConditionalGeneration


model = T5ForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="t5-small",
).to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)

In [5]:
import numpy as np
from torch import nn
from tqdm import tqdm  # tqdm 임포트 추가


def calc_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for source_ids, source_mask, target_ids, target_mask in tqdm(dataloader, desc="Training"):  # tqdm 추가
        decoder_input_ids = target_ids[:, :-1].contiguous()
        labels = target_ids[:, 1:].clone().detach()
        labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

        outputs = model(
            input_ids=source_ids,
            attention_mask=source_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels,
        )

        loss = outputs.loss
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(dataloader)
    return train_loss


def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        val_loss = 0.0

        for source_ids, source_mask, target_ids, target_mask in tqdm(dataloader, desc="Evaluation"):  # tqdm 추가
            decoder_input_ids = target_ids[:, :-1].contiguous()
            labels = target_ids[:, 1:].clone().detach()
            labels[target_ids[:, 1:] == tokenizer.pad_token_id] = -100

            outputs = model(
                input_ids=source_ids,
                attention_mask=source_mask,
                decoder_input_ids=decoder_input_ids,
                labels=labels,
            )

            loss = outputs.loss
            val_loss += loss

    val_loss = val_loss / len(dataloader)
    return val_loss


best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f}")

    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), "./Model/T5.pt")
        print("Saved the model weights")


Training:   0%|          | 1/2792 [00:08<6:19:09,  8.15s/it]


KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    for source_ids, source_mask, target_ids, target_mask in train_dataloader:
        generated_ids = model.generate(
            input_ids=source_ids,
            attention_mask=source_mask,
            max_length=128,
            num_beams=3,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
        )

        for generated, target in zip(generated_ids, target_ids):
            pred = tokenizer.decode(
                generated, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            actual = tokenizer.decode(
                target, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            print("Generated Headline Text:", pred) 
            print("Actual Headline Text   :", actual) 
        break

Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 
Generated Headline Text: FR
Actual Headline Text   : 
Generated Headline Text: 
Actual Headline Text   : 


In [8]:
from transformers import pipeline
model.load_state_dict(torch.load("./Model/T5.pt", map_location=torch.device('cpu')))

summarizer = pipeline(
    task="summarization",
    model=model,
    tokenizer=tokenizer,
    max_length=128,
    device="cpu"
)

for index in range(0,2005):
    news_text = df.text.iloc[index]
    summarization = df.prediction.iloc[index]
    predicted_summarization = summarizer(news_text)[0]["summary_text"]
    print(f"정답 요약문 : {summarization}")
    print(f"모델 요약문 : {predicted_summarization}\n")

Your max_length is set to 128, but your input_length is only 23. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 128, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


정답 요약문 : album
모델 요약문 : nOte,album album,note leaf .account book  ? ?,norty book .



Your max_length is set to 128, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


정답 요약문 : あらまき鮭
모델 요약문 : . ,  ! . & ... ? .,



Your max_length is set to 128, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


...

Your max_length is set to 128, but your input_length is only 22. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


정답 요약문 : トースト
모델 요약문 : yiit81,, . Yiit ii . i.



Your max_length is set to 128, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


정답 요약문 : 山田太郎
모델 요약문 : es, sp,, . ed,: aspp .



KeyboardInterrupt: 